<a href="https://colab.research.google.com/github/cognosx/OpenAlexDataEnricher_notebook/blob/main/OpenAlexDataEnricher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
import os
import pandas as pd
import requests
import re

############# Functions to fetch data ############################

def clean_doi(doi):
    """
    Function to transform DOIs to the required format.
    Converts https://doi.org/10.21203/rs.3.pex-2537/v1 to 10.21203/rs.3.pex-2537/v1.
    """
    if doi and doi.startswith("https://doi.org/"):
        return doi.replace("https://doi.org/", "")
    return doi

# def fetch_openalex_publications(search_query):
#     openalex_url = f'{search_query}'
#     try:
#         response = requests.get(openalex_url)
#         response.raise_for_status()
#         works = response.json().get('results', [])
#         return works
#     except requests.RequestException as e:
#         print(f"Error fetching OpenAlex publications: {e}")
#         return []

# def fetch_openalex_publications(search_query, max_results=100000):
#     works = []
#     per_page = 25
#     page = 1
#     while True:
#         url = f"{search_query}&page={page}"
#         try:
#             response = requests.get(url)
#             response.raise_for_status()
#             data = response.json()
#             results = data.get('results', [])
#             if not results:
#                 break
#             works.extend(results)
#             if len(works) >= max_results:
#                 break
#             page += 1
#         except requests.RequestException as e:
#             print(f"Error fetching OpenAlex publications: {e}")
#             break
#     return works

def fetch_openalex_publications(search_query, max_results=100000):
    works = []
    per_page = 25
    cursor = '*'
    while True:
        url = f"{search_query}&per-page={per_page}&cursor={cursor}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            results = data.get('results', [])
            if not results:
                break
            works.extend(results)
            if len(works) >= max_results:
                break
            cursor = data.get('meta', {}).get('next_cursor')
            if not cursor:
                break
        except requests.RequestException as e:
            print(f"Error fetching OpenAlex publications: {e}")
            break
    return works

def fetch_crossref_metadata(doi):
    doi = clean_doi(doi)
    url = f'https://api.crossref.org/works/{doi}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get('message', {})
    except requests.RequestException as e:
        print(f"Error fetching Crossref metadata: {e}")
        return {}

def fetch_altmetric_data(doi):
    doi = clean_doi(doi)
    altmetric_url = f'https://api.altmetric.com/v1/doi/{doi}'
    try:
        response = requests.get(altmetric_url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching Altmetric data: {e}")
        return {}

def build_publications_dataframe(search_query, include_crossref=False, include_altmetric=True):
    works = fetch_openalex_publications(search_query)
    publications_data = [collect_publication_info(work, include_crossref, include_altmetric) for work in works]
    return pd.DataFrame(publications_data)

def collect_publication_info(work, include_crossref=False, include_altmetric=True):
    doi = work.get('doi', 'Not Available')
    crossref_metadata = fetch_crossref_metadata(doi) if include_crossref and doi != 'Not Available' else {}
    altmetric_data = fetch_altmetric_data(doi) if include_altmetric and doi != 'Not Available' else {}

    institutions_country = []
    institution_type = []
    first_authors_institutions_countries = []
    last_authors_institutions_countries = []
    corresponding_authors_institutions_countries = []

    for authorship in work.get('authorships', []):
        for institution in authorship.get('institutions', []):
            institution_name = institution.get('display_name')
            country_code = institution.get('country_code')
            type_of_institution = institution.get('type')
            institutions_country.append(f"{institution_name} ({country_code})")
            institution_type.append(f"{institution_name} ({type_of_institution})")
            if authorship.get('author_position') == 'first':
                first_authors_institutions_countries.append(f"{institution_name} ({country_code})")
            elif authorship.get('author_position') == 'last':
                last_authors_institutions_countries.append(f"{institution_name} ({country_code})")
            elif authorship.get('is_corresponding') == 'True':
                corresponding_authors_institutions_countries.append(f"{institution_name} ({country_code})")

    institutions_country = list(set(institutions_country))
    institution_type = list(set(institution_type))
    first_authors_institutions_countries = list(set(first_authors_institutions_countries))
    last_authors_institutions_countries = list(set(last_authors_institutions_countries))
    corresponding_authors_institutions_countries = list(set(corresponding_authors_institutions_countries))

    author_countries = set()
    for authorship in work.get('authorships', []):
        for country in authorship.get('countries', []):
            author_countries.add(country)
    author_countries = list(author_countries)

    first_authorships = [authorship for authorship in work.get('authorships', []) if authorship.get('author_position') == 'first']
    last_authorships = [authorship for authorship in work.get('authorships', []) if authorship.get('author_position') == 'last']
    corresponding_authorships = [authorship for authorship in work.get('authorships', []) if authorship.get('is_corresponding')]

    first_authors = [authorship.get('author', {}) for authorship in first_authorships]
    last_authors = [authorship.get('author', {}) for authorship in last_authorships]
    corresponding_authors = [authorship.get('author', {}) for authorship in corresponding_authorships]

    first_author_names = [author.get('display_name') for author in first_authors]
    last_author_names = [author.get('display_name') for author in last_authors]
    corresponding_author_names = [author.get('display_name') for author in corresponding_authors]

    first_authors_countries = [authorship.get('countries', [])[0] if authorship.get('countries') else "Unknown" for authorship in first_authorships]
    last_authors_countries = [authorship.get('countries', [])[0] if authorship.get('countries') else "Unknown" for authorship in last_authorships]
    corresponding_authors_countries = [authorship.get('countries', [])[0] if authorship.get('countries') else "Unknown" for authorship in corresponding_authorships]


    journal = "Not Available"
    j_is_in_doaj = "Not Available"
    primary_location = work.get('primary_location', {})
    if primary_location:
        source = primary_location.get('source', {})
        if source:
            j_is_in_doaj = source.get('is_in_doaj', "Not Available")
            journal = source.get('display_name', "Not Available")
    else:
        source = {}

    apc_list = work.get('apc_list', {}) or {}
    apc_paid = work.get('apc_paid', {}) or {}

    # Initialize the citation counts dictionary
    citation_counts = {}
    for count_data in work.get('counts_by_year', []):
        year = count_data.get('year')
        if year:
            citation_counts[f'Citations {year}'] = count_data.get('cited_by_count', 0)


    publication_info =  {
        'OpenAlex DOI': work.get('doi'),
        'OpenAlex ID': work.get('id'),
        'OpenAlex Title': work.get('display_name'),
        'OpenAlex Publication Year': work.get('publication_year'),
        'OpenAlex Publication Date': work.get('publication_date'),

        'OpenAlex Authors': ', '.join([authorship['author'].get('display_name', 'Unknown Author') for authorship in work.get('authorships', [])]),
        'OpenAlex First Authors': ', '.join(first_author_names),
        'OpenAlex Last Authors': ', '.join(last_author_names),
        'OpenAlex Corresponding Authors': ', '.join(corresponding_author_names),

        'OpenAlex Institutions': ', '.join(institutions_country),
        'OpenAlex Institutions Type': ', '.join(institution_type),
        'OpenAlex First Authors Institutions Countries': ', '.join(first_authors_institutions_countries),
        'OpenAlex Last Authors Institutions Countries': ', '.join(last_authors_institutions_countries),

        'OpenAlex Countries': ', '.join(author_countries),
        'OpenAlex First Authors Countries': ', '.join(first_authors_countries),
        'OpenAlex Last Authors Countries': ', '.join(last_authors_countries),
        'OpenAlex Corresponding Authors Countries': ', '.join(corresponding_authors_countries),

        'number_of_authors': len(work.get('authorships', [])),
        'OpenAlex Institutions Distinct Count': work.get('institutions_distinct_count'),
        'OpenAlex Countries Distinct Count': work.get('countries_distinct_count'),

        'OpenAlex Journal': journal,
        'journal_is_in_doaj': j_is_in_doaj,
        # 'apc_list_value': apc_list.get('value'),
        # 'apc_list_currency': apc_list.get('currency'),
        'apc_list_value_usd': apc_list.get('value_usd'),
        'apc_list_provenance': apc_list.get('provenance'),
        # 'apc_paid_value': apc_paid.get('value'),
        # 'apc_paid_currency': apc_paid.get('currency'),
        'apc_paid_value_usd': apc_paid.get('value_usd'),
        'apc_paid_provenance': apc_paid.get('provenance'),

        'OpenAlex Crossref Publisher': crossref_metadata.get('publisher', ''),
        'OpenAlex Crossref Subject': ', '.join(crossref_metadata.get('subject', [])),
        'OpenAlex topics': ', '.join([topic.get('display_name') for topic in work.get('topics', [])]),
        'OpenAlex Keywords': ', '.join([keyword.get('display_name') for keyword in work.get('keywords', [])]),
        'OpenAlex Mesh': ', '.join([mesh.get('descriptor_name') for mesh in work.get('mesh', [])]),
        'OpenAlex Concepts': ', '.join([concept.get('display_name') for concept in work.get('concepts', [])]),
        'OpenAlex Sustainable Development Goals': ', '.join([sdg.get('display_name') for sdg in work.get('sustainable_development_goals', [])]),

        'OpenAlex Publication Type': work.get('type'),
        'OpenAlex Indexed In': str(work.get('indexed_in')),
        'OpenAlex Language': work.get('language'),

        'OpenAlex Open Access status': work.get('open_access', {}).get('oa_status', None),

        'OpenAlex grants': ', '.join([grant.get('funder_display_name') for grant in work.get('grants', [])]),
        'Crossref Funders': ', '.join(funder.get('name', '') for funder in crossref_metadata.get('funder', [])),

        'Crossref Citation count': crossref_metadata.get('is-referenced-by-count', 0),
        'OpenAlex Cited by Count': work.get('cited_by_count'),
        'OpenAlex Cited Counts by Year': str(work.get('counts_by_year')),

        'Altmetric Score': altmetric_data.get('score'),
        'Altmetric Read Count': altmetric_data.get('readers_count'),
        # 'Altmetric Image': altmetric_data.get('images', {}).get('small'),
        'Altmetric URL': altmetric_data.get('details_url'),
    }


    # Add citation counts to publication_info
    publication_info.update(citation_counts)

    return publication_info


In [ ]:
search_query = 'https://api.openalex.org/works?filter=authorships.institutions.continent:q15,publication_year:2015-2024,title_and_abstract.search:hiv+OR+aids+OR+%22Human+Immunodeficiency+Virus%22+OR+%22HIV/AIDS%22+OR+%22Human+Immunodeficiency+Viruses%22+OR+%22human+immuno-deficiency+virus%22+OR+%22Acquired+Immune+Deficiency+Syndrome%22+OR+%22Acquired+Immunodeficiency+Syndrome%22,type:types/article|types/preprint|types/review|types/letter'
# search_query_test = 'https://api.openalex.org/works?filter=authorships.institutions.continent:q15,publication_year:2024-2024,title_and_abstract.search:hiv+OR+aids+OR+%22Human+Immunodeficiency+Virus%22+OR+%22HIV/AIDS%22+OR+%22Human+Immunodeficiency+Viruses%22+OR+%22human+immuno-deficiency+virus%22+OR+%22Acquired+Immune+Deficiency+Syndrome%22+OR+%22Acquired+Immunodeficiency+Syndrome%22,type:types/review'
publication_data = build_publications_dataframe(search_query)

publication_data.to_csv('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/IMPACT OF COVID-16 ON HIV IN  AFRICA/data/impact_of_covid_in_africa_2014_2024_api.csv')
publication_data.to_excel('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/IMPACT OF COVID-16 ON HIV IN  AFRICA/data/impact_of_covid_in_africa_2014_2024_api.xlsx')
publication_data.to_pickle('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/IMPACT OF COVID-16 ON HIV IN  AFRICA/data/impact_of_covid_in_africa_2014_2024_api.pkl')
publication_data

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.7196/ajhpe.2018.v10i2.543
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.1371/journal.pone.0191967
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.7196/sarj.2018.v24i3.185
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.1177/1556264618783560
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.4103/ijam.ijam_31_18
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.1093/ofid/ofaa439.1836
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.7196/ajhpe.2018.v10i2.543
Error fetching Altmetric 

,OpenAlex DOI,OpenAlex ID,OpenAlex Title,OpenAlex Publication Year,OpenAlex Publication Date,OpenAlex Authors,OpenAlex First Authors,OpenAlex Last Authors,OpenAlex Corresponding Authors,OpenAlex Institutions,OpenAlex Institutions Type,OpenAlex First Authors Institutions Countries,OpenAlex Last Authors Institutions Countries,OpenAlex Countries,OpenAlex First Authors Countries,OpenAlex Last Authors Countries,OpenAlex Corresponding Authors Countries,number_of_authors,OpenAlex Institutions Distinct Count,OpenAlex Countries Distinct Count,OpenAlex Journal,journal_is_in_doaj,apc_list_value_usd,apc_list_provenance,apc_paid_value_usd,apc_paid_provenance,OpenAlex Crossref Publisher,OpenAlex Crossref Subject,OpenAlex topics,OpenAlex Keywords,OpenAlex Mesh,OpenAlex Concepts,OpenAlex Sustainable Development Goals,OpenAlex Publication Type,OpenAlex Indexed In,OpenAlex Language,OpenAlex Open Access status,OpenAlex grants,Crossref Funders,Crossref Citation count,OpenAlex Cited by Count,OpenAlex Cited Counts by Year,Altmetric Score,Altmetric Read Count,Altmetric URL,Citations 2024,Citations 2023,Citations 2022,Citations 2021,Citations 2020,Citations 2019,Citations 2018,Citations 2017,Citations 2016,Citations 2014,Citations 2015,Citations 2012,Citations 2013
0,https://doi.org/10.1016/j.chom.2016.02.011,https://openalex.org/W2296527675,Altered Virome and Bacterial Microbiome in Hum...,2016,2016-03-01,"Cynthia L. Monaco, David B. Gootenberg, Guoyan...",Cynthia L. Monaco,Herbert W. Virgin,"Douglas S. Kwon, Herbert W. Virgin","Washington University in St. Louis (US), Harva...",Washington University in St. Louis (education)...,Washington University in St. Louis (US),Washington University in St. Louis (US),"US, UG",US,US,"US, US",21,7,2,Cell host & microbe,False,9080.0,doaj,9080.0,doaj,,,"Diversity and Function of Gut Microbiome, Huma...","HIV, Obesity-associated Microbiome, AIDS","Acquired Immunodeficiency Syndrome, Acquired I...","Human virome, Biology, Microbiome, Immunodefic...",Good health and well-being,article,"['crossref', 'pubmed']",en,bronze,,,0,315,"[{'year': 2024, 'cited_by_count': 9}, {'year':...",51.63,414.0,https://www.altmetric.com/details.php?citation...,9.0,39.0,38.0,57.0,42.0,48.0,38.0,30.0,14.0,NaN,NaN,NaN,NaN
1,https://doi.org/10.1093/cid/ciy463,https://openalex.org/W2884986477,Human Immunodeficiency Virus Drug Resistance: ...,2018,2018-07-20,"Huldrych F. Günthard, Vincent Cálvez, Roger Pa...",Huldrych F. Günthard,Douglas D. Richman,,"University Hospital of Zurich (CH), University...","Africa Health Research Institute (facility), U...","University Hospital of Zurich (CH), University...","VA San Diego Healthcare System (US), Universit...","GB, US, CH, ES, FR, ZA, NL",CH,US,,8,12,7,Clinical infectious diseases/Clinical infectio...,False,4320.0,doaj,4320.0,doaj,,,Efficacy and Safety of Antiretroviral Therapy ...,"Antiretroviral Therapy, HIV, HIV Epidemiology","Anti-HIV Agents, Drug Resistance, Viral, HIV I...","Medicine, Virology, Drug resistance, Human imm...",,article,"['crossref', 'pubmed']",en,hybrid,,,0,165,"[{'year': 2024, 'cited_by_count': 4}, {'year':...",46.40,272.0,https://www.altmetric.com/details.php?citation...,4.0,21.0,31.0,38.0,41.0,22.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
2,https://doi.org/10.1186/s12879-015-0944-6,https://openalex.org/W1944860507,A systematic review of hepatic tuberculosis wi...,2015,2015-05-06,"Andrew J. Hickey, Lilishia Gounder, Mahomed‐Yu...",Andrew J. Hickey,Paul K. Drain,Andrew J. Hickey,"University of Maryland, Baltimore (US), Massac...","Massachusetts General Hospital (healthcare), H...","University of Maryland, Baltimore (US)","Massachusetts General Hospital (US), Harvard U...","US, ZA",US,US,US,4,4,2,BMC infectious diseases,True,2072.0,doaj,2072.0,doaj,,,Diagnosis and Management of Abdominal Tubercul...,"HIV Infection, Mycobacterium tuberculosis","Acquired Immunodeficiency Syndrome, Mycobacter...","Medicine, Tuberculosis, Internal medicine, Gas...",Good health and well-being,article,"['crossref', 

In [ ]:
publication_data

,OpenAlex DOI,OpenAlex ID,OpenAlex Title,OpenAlex Publication Year,OpenAlex Publication Date,OpenAlex Authors,OpenAlex First Authors,OpenAlex Last Authors,OpenAlex Corresponding Authors,OpenAlex Institutions,OpenAlex Institutions Type,OpenAlex First Authors Institutions Countries,OpenAlex Last Authors Institutions Countries,OpenAlex Countries,OpenAlex First Authors Countries,OpenAlex Last Authors Countries,OpenAlex Corresponding Authors Countries,number_of_authors,OpenAlex Institutions Distinct Count,OpenAlex Countries Distinct Count,OpenAlex Journal,journal_is_in_doaj,apc_list_value_usd,apc_list_provenance,apc_paid_value_usd,apc_paid_provenance,OpenAlex Crossref Publisher,OpenAlex Crossref Subject,OpenAlex topics,OpenAlex Keywords,OpenAlex Mesh,OpenAlex Concepts,OpenAlex Sustainable Development Goals,OpenAlex Publication Type,OpenAlex Indexed In,OpenAlex Language,OpenAlex Open Access status,OpenAlex grants,Crossref Funders,Crossref Citation count,OpenAlex Cited by Count,OpenAlex Cited Counts by Year,Altmetric Score,Altmetric Read Count,Altmetric URL,Citations 2024,Citations 2023,Citations 2022,Citations 2021,Citations 2020,Citations 2019,Citations 2018,Citations 2017,Citations 2016,Citations 2014,Citations 2015,Citations 2012,Citations 2013
0,https://doi.org/10.1016/j.chom.2016.02.011,https://openalex.org/W2296527675,Altered Virome and Bacterial Microbiome in Hum...,2016,2016-03-01,"Cynthia L. Monaco, David B. Gootenberg, Guoyan...",Cynthia L. Monaco,Herbert W. Virgin,"Douglas S. Kwon, Herbert W. Virgin","Washington University in St. Louis (US), Harva...",Washington University in St. Louis (education)...,Washington University in St. Louis (US),Washington University in St. Louis (US),"US, UG",US,US,"US, US",21,7,2,Cell host & microbe,False,9080.0,doaj,9080.0,doaj,,,"Diversity and Function of Gut Microbiome, Huma...","HIV, Obesity-associated Microbiome, AIDS","Acquired Immunodeficiency Syndrome, Acquired I...","Human virome, Biology, Microbiome, Immunodefic...",Good health and well-being,article,"['crossref', 'pubmed']",en,bronze,,,0,315,"[{'year': 2024, 'cited_by_count': 9}, {'year':...",51.63,414.0,https://www.altmetric.com/details.php?citation...,9.0,39.0,38.0,57.0,42.0,48.0,38.0,30.0,14.0,NaN,NaN,NaN,NaN
1,https://doi.org/10.1093/cid/ciy463,https://openalex.org/W2884986477,Human Immunodeficiency Virus Drug Resistance: ...,2018,2018-07-20,"Huldrych F. Günthard, Vincent Cálvez, Roger Pa...",Huldrych F. Günthard,Douglas D. Richman,,"University Hospital of Zurich (CH), University...","Africa Health Research Institute (facility), U...","University Hospital of Zurich (CH), University...","VA San Diego Healthcare System (US), Universit...","GB, US, CH, ES, FR, ZA, NL",CH,US,,8,12,7,Clinical infectious diseases/Clinical infectio...,False,4320.0,doaj,4320.0,doaj,,,Efficacy and Safety of Antiretroviral Therapy ...,"Antiretroviral Therapy, HIV, HIV Epidemiology","Anti-HIV Agents, Drug Resistance, Viral, HIV I...","Medicine, Virology, Drug resistance, Human imm...",,article,"['crossref', 'pubmed']",en,hybrid,,,0,165,"[{'year': 2024, 'cited_by_count': 4}, {'year':...",46.40,272.0,https://www.altmetric.com/details.php?citation...,4.0,21.0,31.0,38.0,41.0,22.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
2,https://doi.org/10.1186/s12879-015-0944-6,https://openalex.org/W1944860507,A systematic review of hepatic tuberculosis wi...,2015,2015-05-06,"Andrew J. Hickey, Lilishia Gounder, Mahomed‐Yu...",Andrew J. Hickey,Paul K. Drain,Andrew J. Hickey,"University of Maryland, Baltimore (US), Massac...","Massachusetts General Hospital (healthcare), H...","University of Maryland, Baltimore (US)","Massachusetts General Hospital (US), Harvard U...","US, ZA",US,US,US,4,4,2,BMC infectious diseases,True,2072.0,doaj,2072.0,doaj,,,Diagnosis and Management of Abdominal Tubercul...,"HIV Infection, Mycobacterium tuberculosis","Acquired Immunodeficiency Syndrome, Mycobacter...","Medicine, Tuberculosis, Internal medicine, Gas...",Good health and well-being,article,"['crossref', 